# Agents
Intelligent systems that utilize LLMs to determine actions and facilitate complex tasks, allowing LLMs to behave as **reasoning engines** ratehr than simply a **content generator**. They combine tools with LLM.

We will see the following:
- Introduction
    - LLMs as reasoning engines
    - LLMs as content generators
- Built in LangChain tools
    - Wikipedia and llm-math
    - Google Search
    - Requests
    - Python-REPL
- Python Agent
- Defining your own tool
---

## Setup

In [1]:
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
openai.api_type = os.environ.get("OPENAI_API_TYPE")
openai.api_base = os.environ.get("OPENAI_API_BASE")
openai.api_key = os.environ.get("OPENAI_API_KEY")
openai.api_version = os.environ.get("OPENAI_API_VERSION")

In [3]:
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt4",
    temperature=0,
)

## Introduction 

In the context of language models, agents are used to decide the course of action and the sequence of these actions. These actions can either be the utilization of a tool, observing its output, or offering a response to the user. The real potential of agents unfolds when they are utilized appropriately. This explanation aims to simplify the usage of agents via the highest-level API.

Before diving into the practical usage, it's crucial to understand the following terms:
1. **Tool**: A function that performs a specific task. It can be a Google Search, a Database lookup, a Python REPL, or other chains. A tool's interface is typically a function that takes a string as an input and returns a string as an output.
2. **Language Learning Model**: The language model that powers the agent.
3. **Agent**: The agent to use, identified by a string that references a supported agent class. It’s what orchestrates the LLM and the tools. This explanation focuses on using the standard supported agents via the highest-level API. For custom agent implementation, refer to the appropriate documentation.

Agents in LangChain play a crucial role in the decision-making and execution of tasks based on user input. They evaluate the situation and decide on the appropriate tools to use, if necessary.

Presently, most of the agents in LangChain fall into one of these two categories:
- **Action Agents**: These agents determine and execute a single action. They are typically used for straightforward tasks.
- **Plan-and-Execute Agents**: These agents first devise a plan comprising multiple actions and then execute each action sequentially. They are more suited for complex or long-running tasks as they help maintain focus on long-term objectives.
While Action Agents are more traditional and suitable for smaller tasks, Plan-and-Execute Agents help maintain long-term objectives and focus. However, they might lead to more calls and higher latency. Often, it's beneficial to let an Action Agent manage the execution for the Plan-and-Execute agent, thus utilizing both strengths.

For example, a high-level workflow of Action Agents would look something like this:
1. The agent receives user input.
2. It decides which tool to use (if any) and determines its input.
3. The chosen tool is called with the provided input, and an observation (the output of the tool) is recorded.
4. The history of the tool, tool input, and observation are relayed back to the agent, which then decides the next step.
5. This process is repeated until the agent no longer needs to use a tool, at which point it directly responds to the user.

The most critical abstraction to understand is the agent itself. In the context of LangChain, the term "agents" pertains to the concept of employing a language model as a reasoning mechanism and linking it with the key element - a tool.

Tools are instrumental in connecting the language model with other sources of data or computation, including search engines, APIs, and other data repositories. Language models can only access the knowledge they've been trained on, which can quickly become obsolete. Therefore, tools are essential as they allow the agent to retrieve and incorporate current data into the prompt as context. Tools can also execute actions (like running code or modifying files) and observe the results, subsequently informing the language model's decision-making process.

As we said before, we can abstract two primary modes of operation to consider when employing an LLM: as a content generator and as a reasoning engine.
- When used as a "content generator," the language model is asked to create content entirely from its internal knowledge base. This approach can lead to highly creative outputs but can also result in unverified information or 'hallucinations' due to the model's reliance on pre-trained knowledge.
- On the other hand, when functioning as a "reasoning engine," the Agent acts more as an information manager rather than a creator. In this mode, it is tasked with gathering relevant, accurate information, often aided by external tools. This involves the LLM drawing from similar resources on a given topic and constructing new content by extracting and summarizing the relevant details.

### LLMs as reasoning engines

In the example, the agent leverages its "reasoning engine" capabilities to generate responses. Instead of creating new content (acting as a content generator), the agent uses the tools at its disposal to gather, process, and synthesize information. The entire output was truncated, and the agent skillfully employed the LLM-math tool.

In [23]:
# Importing necessary modules
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType
from langchain.chat_models import AzureChatOpenAI

# Loading the language model to control the agent
llm = AzureChatOpenAI(deployment_name="gpt4", temperature=0)

# Loading some tools to use. The llm-math tool uses an LLM, so we pass that in.
tools = load_tools(["google-search", "llm-math"], llm=llm)

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Testing the agent
query = "What's the result of 1000 plus the number of goals scored in the soccer world cup in 2018?"
response = agent.run(query)
print(response)



> Entering new  chain...
Thought: I need to find the number of goals scored in the soccer world cup in 2018.
Action:
```
{
  "action": "google_search",
  "action_input": "number of goals scored in the soccer world cup 2018"
}
```
Observation: Jan 13, 2023 ... Total number of goals scored at each FIFA World Cup from 1930 to 2022 ... Germany 2006 South Africa 2010 Brazil 2014 Russia 2018 Qatar 2022. Jan 31, 2020 ... A total of 169 goals were scored at the group and knockout stages of the FIFA World Cup held in Russia from the 14th of June to the 15th of July ... Jan 13, 2023 ... This number can be attributed to some unusually high-scoring games, which included Austria 7 Switzerland 5 and Hungary 9 South Korea 0. The ... Number of goals scored in the matches played between the teams in question;; Fair play points in all group matches (only one deduction could be applied to a ... Goals and performances from midfielder Paul Pogba and forward Antoine Griezmann also proved vital in helping 

### LLMs as content generators

Here, the Agent is primarily using its internal knowledge to generate the output. Here's a brief explanation of how that works:
- The agent receives a prompt to "Compose an epic science fiction saga about interstellar explorers.”
- The agent then uses its understanding of language, narrative structure, and the specific themes mentioned in the prompt (science fiction, interstellar exploration, etc.) to generate a story.

In [25]:
# Importing necessary modules
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import Tool
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["query"],
    template="You're a renowned science fiction writer. {query}",
)

# Initialize the language model
llm = AzureChatOpenAI(deployment_name="gpt4", temperature=1)
llm_chain = LLMChain(llm=llm, prompt=prompt)

tools = [
    Tool(
        name="Science Fiction Writer",
        func=llm_chain.run,
        description="Use this tool for generating science fiction stories. \
Input should be a command about generating specific types of stories.",
    )
]

# Initializing an agent with the tools, the language model, and the type of agent we want to use.
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

# Testing the agent with the new prompt
response = agent.run(
    "Compose an epic science fiction saga about interstellar explorers"
)
print(response)



> Entering new  chain...
Thought: I should use the Science Fiction Writer tool to generate an epic science fiction saga about interstellar explorers.
Action:
```
{
  "action": "Science Fiction Writer",
  "action_input": "Compose an epic science fiction saga about interstellar explorers"
}
```
Observation: Title: The Celestial Odyssey

Book One: Echoes of Celestial Triumph

In a distant future, where humanity has advanced far beyond the borders of the Milky Way, the vast reaches of the cosmos have become the new frontiers of exploration. Through the advancement of AI singularity and breakthroughs in hyperspace technology, cosmic travel had become a common reality. However, there remained a single, impenetrable barrier that eluded even the most determined scholars – the enigmatic Veil, a region of collapsed star systems and black holes that had devoured everything that dared approach it.

In the heart of the Galactic Commonwealth, the United Alliance of Human Colonies (UAHC), a steadfa

## Built-in LangChain tools

### Wikipedia and llm-math

In [15]:
from langchain.agents import load_tools, initialize_agent
from langchain.agents import AgentType

In [16]:
tools = load_tools(["llm-math", "wikipedia"], llm=llm)

In [17]:
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

In [18]:
agent("What is the 25% of 300?")



> Entering new  chain...
Thought: I need to calculate 25% of 300.
Action:
```
{
  "action": "Calculator",
  "action_input": "0.25 * 300"
}
```
Observation: Answer: 75.0
Thought:I now know the final answer
Final Answer: 75

> Finished chain.


{'input': 'What is the 25% of 300?', 'output': '75'}

In [19]:
question = "Tom M. Mitchell is an American computer scientist \
and the Founders University Professor at Carnegie Mellon University (CMU)\
what book did he write?"

result = agent(question)
result



> Entering new  chain...
Thought: I need to find the book written by Tom M. Mitchell.
Action:
```
{
  "action": "Wikipedia",
  "action_input": "Tom M. Mitchell"
}
```
Observation: Page: Tom M. Mitchell
Summary: Tom Michael Mitchell (born August 9, 1951) is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU). He is a founder and former Chair of the Machine Learning Department at CMU. Mitchell is known for his contributions to the advancement of machine learning, artificial intelligence, and cognitive neuroscience and is the author of the textbook Machine Learning. He is a member of the United States National Academy of Engineering since 2010. He is also a Fellow of the American Academy of Arts and Sciences, the American Association for the Advancement of Science and a Fellow and past President of the Association for the Advancement of Artificial Intelligence. In October 2018, Mitchell was appointed as the Interim Dean of the School 

{'input': 'Tom M. Mitchell is an American computer scientist and the Founders University Professor at Carnegie Mellon University (CMU)what book did he write?',
 'output': 'Tom M. Mitchell is the author of the textbook "Machine Learning."'}

### Google-Search

We must set up the API Key and a custom search engine to be able to use Google search API. To get the key, head to the [Google Cloud console](https://console.cloud.google.com/apis/credentials) and generate the key by pressing the CREATE CREDENTIALS buttons from the top and choosing API KEY. Then, head to the [Programmable Search Engine](https://programmablesearchengine.google.com/controlpanel/create) dashboard and remember to select the “Search the entire web” option. The Search engine ID will be visible in the details. You might also need to enable the “Custom Search API” service under the Enable APIs and services. (You will receive the instruction from API if required) We can now configure the environment variables `GOOGLE_CSE_ID` and `GOOGLE_API_KEY`, allowing the Google wrapper to connect with the API.

#### As a standalone utility

In [4]:
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
search.results("What is the capital of Spain?", 3)

[{'title': 'Madrid - Wikipedia',
  'link': 'https://en.wikipedia.org/wiki/Madrid',
  'snippet': 'Madrid is the capital and most populous city of Spain. The city has almost 3.4 million inhabitants and a metropolitan area population of approximately 6.7\xa0...'},
 {'title': 'Madrid | History, Population, Climate, & Facts | Britannica',
  'link': 'https://www.britannica.com/place/Madrid',
  'snippet': "Jun 28, 2023 ... Madrid, city, capital of Spain and of Madrid provincia (province). Spain's arts and financial centre, the city proper and province form a\xa0..."},
 {'title': 'Madrid - Eurocities',
  'link': 'https://eurocities.eu/cities/madrid/',
  'snippet': 'As the Spanish capital, Madrid is home to embassies and international organisations, major companies and financial institutions. It ranks first in Spain for the\xa0...'}]

#### With Agent

In [6]:
from langchain.agents import initialize_agent, load_tools, AgentType

tools = load_tools(["google-search"])

agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

print(agent("What is the national sweet of India?"))



> Entering new  chain...
I am not sure about the national sweet of India. I should search for it using google_search.
Action: google_search
Action Input: national sweet of India
Observation: Jul 29, 2013 ... Do you know that Jalebi is the national sweet of India and you will find this sweet in every corner of India and the taste of the Jalebis of ... Jalebi is India's national sweet and is available in any corner of India. But being a foodie there are many others which are worthy of mention. Gulab Jamun. According to the Indian ambassador Nagma Malik, jalebi might have started life in Turkey and then arrived in Tunisia long ago before making its way to India ... Jalebi and its Indian cousins: Jalebi while famous as a North Indian sweet also became renowned in the South by the name Jilebi and Jilapi in Bengal. Jul 29, 2013 ... Do you know that Jalebi is the national sweet of India and you will find this sweet in every corner of India and the taste of the Jalebis of ... Dec 21, 2022 ..

### Requests

Let's do a manual request to an API and see the reposnses.

In [9]:
import requests

response = requests.get(
    "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
).json()
response

{'data': [{'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2020,
   'Year': '2020',
   'Population': 326569308,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2019,
   'Year': '2019',
   'Population': 324697795,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2018,
   'Year': '2018',
   'Population': 322903030,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2017,
   'Year': '2017',
   'Population': 321004407,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2016,
   'Year': '2016',
   'Population': 318558162,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'Nation': 'United States',
   'ID Year': 2015,
   'Year': '2015',
   'Population': 316515021,
   'Slug Nation': 'united-states'},
  {'ID Nation': '01000US',
   'N

Let's use the tool to do the same now.

In [20]:
from langchain.agents import AgentType

tools = load_tools(["requests_all"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

query = "Get the response from \
https://datausa.io/api/data?drilldowns=Nation&measures=Population."

response = agent.run(query)
response



> Entering new  chain...
Thought: I need to use the requests_get tool to get the response from the provided URL.

Action:
```
{
  "action": "requests_get",
  "action_input": "https://datausa.io/api/data?drilldowns=Nation&measures=Population"
}
```
Observation: {"data":[{"ID Nation":"01000US","Nation":"United States","ID Year":2020,"Year":"2020","Population":326569308,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2019,"Year":"2019","Population":324697795,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2018,"Year":"2018","Population":322903030,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2017,"Year":"2017","Population":321004407,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2016,"Year":"2016","Population":318558162,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2015,"Year":"20

'{"data":[{"ID Nation":"01000US","Nation":"United States","ID Year":2020,"Year":"2020","Population":326569308,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2019,"Year":"2019","Population":324697795,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2018,"Year":"2018","Population":322903030,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2017,"Year":"2017","Population":321004407,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2016,"Year":"2016","Population":318558162,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2015,"Year":"2015","Population":316515021,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2014,"Year":"2014","Population":314107084,"Slug Nation":"united-states"},{"ID Nation":"01000US","Nation":"United States","ID Year":2013,"Year":"20

### Python-REPL

In [22]:
tools = load_tools(["python_repl"], llm=llm)
agent = initialize_agent(
    tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

query = "Create a list of random strings containing 4 letters, \
list should contain 10 examples, and sort the list alphabetically"

print(agent.run(query))



> Entering new  chain...
Thought: I will use the `random` module to generate random strings and then sort the list alphabetically.

Action:
```
{
  "action": "Python_REPL",
  "action_input": "import random; import string; random_strings = [''.join(random.choices(string.ascii_letters, k=4)) for _ in range(10)]; sorted_strings = sorted(random_strings); print(sorted_strings)"
}
```
Observation: ['AvOp', 'LuQI', 'RNvn', 'VYyS', 'VZQU', 'nPCa', 'oNGn', 'qLUJ', 'yEVA', 'yfwF']

Thought:I now know the final answer.
Final Answer: ['AvOp', 'LuQI', 'RNvn', 'VYyS', 'VZQU', 'nPCa', 'oNGn', 'qLUJ', 'yEVA', 'yfwF']

> Finished chain.
['AvOp', 'LuQI', 'RNvn', 'VYyS', 'VZQU', 'nPCa', 'oNGn', 'qLUJ', 'yEVA', 'yfwF']


## Python Agent
`PythonREPLTool` does not work correctly with Azure deployed "GPT-4" model right now. Please use `ChatOpenAI` class instead of `AzureChatOpenAI` for the following example.

We will reset the env variables, **please restart the kernel before running the cells below**.

In [1]:
# Reset the env vars
import openai
import os
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
# openai.api_type = os.environ.get("OPENAI_API_TYPE")
# openai.api_base = os.environ.get("OPENAI_API_BASE")
# openai.api_version = os.environ.get("OPENAI_API_VERSION")
openai.api_key = os.environ.get("OPENAI_API_KEY")

In [2]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.chat_models import ChatOpenAI

agent = create_python_agent(
    llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
    tool=PythonREPLTool(),
    verbose=True,
    handle_parsing_errors=True,
)

In [3]:
customer_list = [
    ["Harrison", "Chase"],
    ["Lang", "Chain"],
    ["Dolly", "Too"],
    ["Elle", "Elem"],
    ["Geoff", "Fusion"],
    ["Trance", "Former"],
    ["Jen", "Ayai"],
]

In [4]:
agent.run(
    f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""
)



> Entering new  chain...
I can use the `sorted()` function to sort the list of customers. I will need to provide a key function that specifies the sorting order based on last name and then first name.
Action: Python_REPL
Action Input: sorted([['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']], key=lambda x: (x[1], x[0]))
Observation: 
Thought:The customers have been sorted by last name and then first name.
Final Answer: [['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]

> Finished chain.


"[['Jen', 'Ayai'], ['Harrison', 'Chase'], ['Lang', 'Chain'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Dolly', 'Too']]"

In [5]:
# View detailed outputs of the chains

import langchain

langchain.debug = True
agent.run(
    f"""Sort these customers by \
last name and then first name \
and print the output: {customer_list}"""
)
langchain.debug = False

[chain/start] [1:RunTypeEnum.chain:AgentExecutor] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]"
}
[chain/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain] Entering Chain run with input:
{
  "input": "Sort these customers by last name and then first name and print the output: [['Harrison', 'Chase'], ['Lang', 'Chain'], ['Dolly', 'Too'], ['Elle', 'Elem'], ['Geoff', 'Fusion'], ['Trance', 'Former'], ['Jen', 'Ayai']]",
  "agent_scratchpad": "",
  "stop": [
    "\nObservation:",
    "\n\tObservation:"
  ]
}
[llm/start] [1:RunTypeEnum.chain:AgentExecutor > 2:RunTypeEnum.chain:LLMChain > 3:RunTypeEnum.llm:ChatOpenAI] Entering LLM run with input:
{
  "prompts": [
    "Human: You are an agent designed to write and execute python code to answer questions.\nYou ha

## Define your own tool

In [23]:
from langchain.agents import tool
from datetime import date

In [24]:
@tool
def time(text: str) -> str:
    """Returns todays date, use this for any \
    questions related to knowing todays date. \
    The input should always be an empty string, \
    and this function will always return todays \
    date - any date mathmatics should occur \
    outside this function."""
    return str(date.today())

In [25]:
agent = initialize_agent(
    tools + [time],
    llm,
    agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    handle_parsing_errors=True,
    verbose=True,
)

> Note:
The agent will sometimes come to the wrong conclusion (agents are a work in progress!). If it does, please try running it again.

In [28]:
try:
    result = agent("whats the date today?")
    print(result)
except:
    print("exception on external access")



> Entering new  chain...
Thought: I need to find today's date.
Action:
```
{
  "action": "time",
  "action_input": ""
}
```

Observation: 2023-06-28
Thought:I now know the final answer
Final Answer: 2023-06-28

> Finished chain.
{'input': 'whats the date today?', 'output': '2023-06-28'}
